In [1]:
from pathlib import Path
from bids import BIDSLayout
from contarg.utils import make_rel_symlink
import json
import subprocess
import nibabel as nb
import scipy.io as sio
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

# make a symlinked liston input directory structure

In [39]:
bids_dir = Path('/data/EDB/TMSpilot/BIDS')
liston_dir = Path('/data/EDB/TMSpilot/liston')
liston_dir.mkdir(exist_ok=True)

In [40]:
subjects = ['24546']


In [41]:
subjects

['24563', '24573', '24704', '24718', '24740']

In [41]:
layout = BIDSLayout(bids_dir)

In [42]:
fmap_dir_map = {
    'j-':'AP',
    'j':'PA'
}

In [44]:
subjects

['24546']

In [43]:
# create input directory
for subject in subjects:
    lsub_dir = liston_dir / f'sub-{subject}'
    lsub_dir.mkdir(exist_ok=True)
    anat_dir = lsub_dir / 'anat'
    t1w_dir = anat_dir / 'unprocessed/T1w'
    t1w_dir.mkdir(exist_ok=True, parents=True)
    t2w_dir = anat_dir / 'unprocessed/T2w'
    t2w_dir.mkdir(exist_ok=True, parents=True)
    t1ws = layout.get(return_type='file', subject=subject, datatype='anat', suffix='T1w', extension='.nii.gz')
    t1ws = [tt for tt in t1ws if not 'norm' in tt]
    for ix, t1w in enumerate(t1ws):
        lpath = t1w_dir /f'T1w_{ix + 1}.nii.gz'
        make_rel_symlink(lpath, Path(t1w))

    t2ws = layout.get(return_type='file', subject=subject, datatype='anat', suffix='T2w', extension='.nii.gz')
    t2ws = [tt for tt in t2ws if not 'norm' in tt]
    for ix, t2w in enumerate(t2ws):
        lpath = t2w_dir /f'T2w_{ix + 1}.nii.gz'
        make_rel_symlink(lpath, Path(t2w))
        
    func_dir = lsub_dir / 'func/unprocessed/rest'
    func_dir.mkdir(exist_ok=True, parents=True)
    fmap_dir = lsub_dir / 'func/unprocessed/field_maps'
    fmap_dir.mkdir(exist_ok=True, parents=True)
    
    # make fmap_dict
    field_maps = layout.get(subject=subject, datatype='fmap', extension='.nii.gz')
    fmap_lut = {}
    for fmap in field_maps:
        fmap_lut[fmap.get_metadata()['B0FieldIdentifier']] = {}
    for fmap in field_maps:
        fmap_lut[fmap.get_metadata()['B0FieldIdentifier']][fmap_dir_map[fmap.get_metadata()['PhaseEncodingDirection']]] = fmap
    for session in layout.get(return_type='id', target='session', subject=subject):
        ses_dir = func_dir/f'session_{session}'
        lrun = 1
        for brun in layout.get(return_type='id', target='run', subject=subject, session=session):
            if (subject == '24704') & (brun == '1') & (session == '1'):
                continue
            me_bolds = layout.get(subject=subject,
                                  session=session,
                                  run=brun,
                                  datatype='func', acquisition='MBME', extension='.nii.gz')
            run_dir = ses_dir / f'run_{lrun}'
            run_dir.mkdir(exist_ok=True, parents=True)
            for bold in me_bolds:
                echo = bold.get_entities()['echo']
                bjson = bold.get_associations()[0]
                lpath = run_dir / f'Rest_S{session}_R{lrun}_E{echo}.nii.gz'
                make_rel_symlink(lpath, Path(bold.path))
                lbjson_path = run_dir / f'Rest_S{session}_R{lrun}_E{echo}.json'
                make_rel_symlink(lbjson_path, Path(bjson.path))

                if echo == '1':
                    fsource = bold.get_metadata()['B0FieldSource']
                    lapfmap_path = fmap_dir / f'AP_S{session}_R{lrun}.nii.gz'
                    make_rel_symlink(lapfmap_path, Path(fmap_lut[fsource]['AP'].path))
                    lapfmapjson_path = fmap_dir / f'AP_S{session}_R{lrun}.json'
                    make_rel_symlink(lapfmapjson_path, Path(fmap_lut[fsource]['AP'].get_associations()[0].path))
                    try:
                        lpafmap_path = fmap_dir / f'PA_S{session}_R{lrun}.nii.gz'
                        make_rel_symlink(lpafmap_path, Path(fmap_lut[fsource]['PA'].path))
                        lpafmapjson_path = fmap_dir / f'PA_S{session}_R{lrun}.json'
                        make_rel_symlink(lpafmapjson_path, Path(fmap_lut[fsource]['PA'].get_associations()[0].path))
                    except KeyError:
                        # if the PA doesn't exist, we'll have to make it
                        cmd = [
                            '3dcalc',
                            '-a',
                            f"{bold.path}[0..9]",
                            '-expr',
                            "a",
                            '-prefix',
                            lpafmap_path.as_posix(),
                            '-overwrite'
                        ]
                        subprocess.run(cmd, check=True)
                        lpafmapjson_path = fmap_dir / f'PA_S{session}_R{lrun}.json'
                        fmap_json_dat = json.loads(lapfmapjson_path.read_text())
                        fmap_json_dat['PhaseEncodingDirection'] = 'j'
                        lpafmapjson_path.write_text(json.dumps(fmap_json_dat, indent=2))
            lrun += 1

In [43]:
# mv sub-24718_ses-3 to a second subject for liston purposes
new_ses_3_dir = liston_dir/'sub-247183'
new_ses_3_dir.mkdir(exist_ok=True)
old_sub_dir =  liston_dir/'sub-24718'

# mv unprocessed rest to new directory
new_rest_dir = new_ses_3_dir / 'func/unprocessed/rest/session_1'
old_rest_dir = old_sub_dir / 'func/unprocessed/rest/session_3'
try:
    new_rest_dir.parent.mkdir(exist_ok=True, parents=True)
    old_rest_dir.rename(new_rest_dir)
except FileNotFoundError:
    pass
# rename rest files
new_rest_dir_oldfiles = sorted(new_rest_dir.glob('*/Rest_*.*'))
for ff in new_rest_dir_oldfiles:
    new_ff = ff.as_posix().replace('_S3_', '_S1_')
    ff.rename(new_ff)
    
# make new fieldmap directory
old_fm_dir = old_sub_dir / 'func/unprocessed/field_maps'
new_fm_dir = new_ses_3_dir / 'func/unprocessed/field_maps'
new_fm_dir.mkdir(exist_ok=True)
# rename fieldmaps
old_s3_fms = sorted(old_fm_dir.glob('*_S3_*.*'))
for ff in old_s3_fms:
    new_ff = new_fm_dir / ff.parts[-1].replace('_S3_', '_S1_')
    ff.rename(new_ff)



In [90]:
# create directories with mbfieldmaps for 247183, 24740, and 24742
altbids_dir = Path('/data/EDB/TMSpilot/BIDS.alternatefmap')

altlayout = BIDSLayout(altbids_dir)

In [100]:
liston_dir = Path('/data/EDB/TMSpilot/liston')


In [93]:
subjects = ['24718', '24740', '24742']
# create input directory
for subject in subjects:
    lsub_dir = liston_dir / f'sub-{subject}sbref'
    lsub_dir.mkdir(exist_ok=True)
    anat_dir = lsub_dir / 'anat'
    t1w_dir = anat_dir / 'unprocessed/T1w'
    t1w_dir.mkdir(exist_ok=True, parents=True)
    t2w_dir = anat_dir / 'unprocessed/T2w'
    t2w_dir.mkdir(exist_ok=True, parents=True)
    t1ws = altlayout.get(return_type='file', subject=subject, datatype='anat', suffix='T1w', extension='.nii.gz')
    t1ws = [tt for tt in t1ws if not 'norm' in tt]
    for ix, t1w in enumerate(t1ws):
        lpath = t1w_dir /f'T1w_{ix + 1}.nii.gz'
        make_rel_symlink(lpath, Path(t1w))

    t2ws = altlayout.get(return_type='file', subject=subject, datatype='anat', suffix='T2w', extension='.nii.gz')
    t2ws = [tt for tt in t2ws if not 'norm' in tt]
    for ix, t2w in enumerate(t2ws):
        lpath = t2w_dir /f'T2w_{ix + 1}.nii.gz'
        make_rel_symlink(lpath, Path(t2w))
        
    func_dir = lsub_dir / 'func/unprocessed/rest'
    func_dir.mkdir(exist_ok=True, parents=True)
    fmap_dir = lsub_dir / 'func/unprocessed/field_maps'
    fmap_dir.mkdir(exist_ok=True, parents=True)
    
    # make fmap_dict
    field_maps = altlayout.get(subject=subject, datatype='fmap', extension='.nii.gz')
    fmap_lut = {}
    for fmap in field_maps:
        fmap_lut[fmap.get_metadata()['B0FieldIdentifier']] = {}
    for fmap in field_maps:
        fmap_lut[fmap.get_metadata()['B0FieldIdentifier']][fmap_dir_map[fmap.get_metadata()['PhaseEncodingDirection']]] = fmap
    for session in altlayout.get(return_type='id', target='session', subject=subject):
        ses_dir = func_dir/f'session_{session}'
        lrun = 1
        for brun in altlayout.get(return_type='id', target='run', subject=subject, session=session):
            if (subject == '24704') & (brun == '1') & (session == '1'):
                continue
            me_bolds = altlayout.get(subject=subject,
                                  session=session,
                                  run=brun,
                                  datatype='func', acquisition='MBME', extension='.nii.gz')
            run_dir = ses_dir / f'run_{lrun}'
            run_dir.mkdir(exist_ok=True, parents=True)
            for bold in me_bolds:
                echo = bold.get_entities()['echo']
                bjson = bold.get_associations()[0]
                lpath = run_dir / f'Rest_S{session}_R{lrun}_E{echo}.nii.gz'
                make_rel_symlink(lpath, Path(bold.path))
                lbjson_path = run_dir / f'Rest_S{session}_R{lrun}_E{echo}.json'
                make_rel_symlink(lbjson_path, Path(bjson.path))

                if echo == '1':
                    fsource = bold.get_metadata()['B0FieldSource']
                    lapfmap_path = fmap_dir / f'AP_S{session}_R{lrun}.nii.gz'
                    make_rel_symlink(lapfmap_path, Path(fmap_lut[fsource]['AP'].path))
                    lapfmapjson_path = fmap_dir / f'AP_S{session}_R{lrun}.json'
                    make_rel_symlink(lapfmapjson_path, Path(fmap_lut[fsource]['AP'].get_associations()[0].path))
                    try:
                        lpafmap_path = fmap_dir / f'PA_S{session}_R{lrun}.nii.gz'
                        make_rel_symlink(lpafmap_path, Path(fmap_lut[fsource]['PA'].path))
                        lpafmapjson_path = fmap_dir / f'PA_S{session}_R{lrun}.json'
                        make_rel_symlink(lpafmapjson_path, Path(fmap_lut[fsource]['PA'].get_associations()[0].path))
                    except KeyError:
                        # if the PA doesn't exist, we'll have to make it
                        cmd = [
                            '3dcalc',
                            '-a',
                            f"{bold.path}[0..9]",
                            '-expr',
                            "a",
                            '-prefix',
                            lpafmap_path.as_posix(),
                            '-overwrite'
                        ]
                        subprocess.run(cmd, check=True)
                        lpafmapjson_path = fmap_dir / f'PA_S{session}_R{lrun}.json'
                        fmap_json_dat = json.loads(lapfmapjson_path.read_text())
                        fmap_json_dat['PhaseEncodingDirection'] = 'j'
                        lpafmapjson_path.write_text(json.dumps(fmap_json_dat, indent=2))
            lrun += 1

++ 3dcalc: AFNI version=AFNI_23.2.08 (Aug 22 2023) [64-bit]
++ Authored by: A cast of thousands
++ WARNING: file /home/nielsond/.afni.log is now 865859908 (866 million) bytes long!
++ Output dataset /data/EDB/TMSpilot/liston/sub-24718sbref/func/unprocessed/field_maps/PA_S1_R1.nii.gz
++ 3dcalc: AFNI version=AFNI_23.2.08 (Aug 22 2023) [64-bit]
++ Authored by: A cast of thousands
++ WARNING: file /home/nielsond/.afni.log is now 865860173 (866 million) bytes long!
++ Output dataset /data/EDB/TMSpilot/liston/sub-24718sbref/func/unprocessed/field_maps/PA_S1_R2.nii.gz
++ 3dcalc: AFNI version=AFNI_23.2.08 (Aug 22 2023) [64-bit]
++ Authored by: A cast of thousands
++ WARNING: file /home/nielsond/.afni.log is now 865860438 (866 million) bytes long!
++ Output dataset /data/EDB/TMSpilot/liston/sub-24718sbref/func/unprocessed/field_maps/PA_S1_R3.nii.gz
++ 3dcalc: AFNI version=AFNI_23.2.08 (Aug 22 2023) [64-bit]
++ Authored by: A cast of thousands
++ WARNING: file /home/nielsond/.afni.log is now 865

In [94]:
# mv sub-24718_ses-3 to a second subject for liston purposes
new_ses_3_dir = liston_dir/'sub-247183sbref'
new_ses_3_dir.mkdir(exist_ok=True)
old_sub_dir =  liston_dir/'sub-24718sbref'

# mv unprocessed rest to new directory
new_rest_dir = new_ses_3_dir / 'func/unprocessed/rest/session_1'
old_rest_dir = old_sub_dir / 'func/unprocessed/rest/session_3'
try:
    new_rest_dir.parent.mkdir(exist_ok=True, parents=True)
    old_rest_dir.rename(new_rest_dir)
except FileNotFoundError:
    pass
# rename rest files
new_rest_dir_oldfiles = sorted(new_rest_dir.glob('*/Rest_*.*'))
for ff in new_rest_dir_oldfiles:
    new_ff = ff.as_posix().replace('_S3_', '_S1_')
    ff.rename(new_ff)
    
# make new fieldmap directory
old_fm_dir = old_sub_dir / 'func/unprocessed/field_maps'
new_fm_dir = new_ses_3_dir / 'func/unprocessed/field_maps'
new_fm_dir.mkdir(exist_ok=True)
# rename fieldmaps
old_s3_fms = sorted(old_fm_dir.glob('*_S3_*.*'))
for ff in old_s3_fms:
    new_ff = new_fm_dir / ff.parts[-1].replace('_S3_', '_S1_')
    ff.rename(new_ff)

# manually copy the anat directory over

# run hires HCP wrapper
example comand:   
`./anat_highres_HCP_wrapper_par.sh /data/EDB/TMSpilot/liston sub-24704 10`

In [45]:
anat_script_path = '/data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/anat_highres_HCP_wrapper_par.sh'

swarm_cmd_dir = liston_dir/'swarm/swarm_cmds'
swarm_cmd_dir.mkdir(exist_ok=True, parents=True)
swarm_log_dir = liston_dir/'swarm/swarm_log'
swarm_log_dir.mkdir(exist_ok=True, parents=True)

In [46]:
nthreads = 20
cmds = []
for sub in  ['24546']:
    cmd = [
        'bash',
        anat_script_path,
        liston_dir.as_posix(),
        f'sub-{sub}',
        f'{nthreads}'
    ]
    cmds.append(' '.join(cmd))

In [47]:
cmds

['bash /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/anat_highres_HCP_wrapper_par.sh /data/EDB/TMSpilot/liston sub-24546 20']

In [48]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'liston_anat'
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'liston_anat'
    jobid = ! swarm -f {swarm_cmd_file} -g 80 -t 22 --gres=lscratch:200 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm
    print(jobid)

['12206781']


## check hires outputs

In [31]:
anat_success = []
for sub in subjects + ['247183']:
    row = dict(subject=sub)
    for fn in ['PreFreeSurfer', 'FreeSurfer', 'PostFreeSurfer']:
        ff = liston_dir / f'sub-{sub}/anat/qa/{fn}.txt'
        flog = ff.read_text()
        row[fn] = 'Completed!' in flog.split('\n')[-2]
    anat_success.append(row)
anat_success = pd.DataFrame(anat_success)

In [32]:
anat_success

,subject,PreFreeSurfer,FreeSurfer,PostFreeSurfer
0,24563,True,True,True
1,24573,True,True,True
2,24704,True,True,True
3,24718,True,True,True
4,24740,True,True,True
5,247183,True,True,True


# run func_preproc+denoise
./func_preproc+denoise_ME-fMRI_wrapper.sh /data/EDB/TMSpilot/liston sub-24704 15

In [53]:
func_script_path = '/data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/func_preproc+denoise_ME-fMRI_wrapper.sh'

swarm_cmd_dir = liston_dir/'swarm/swarm_cmds'
swarm_cmd_dir.mkdir(exist_ok=True, parents=True)
swarm_log_dir = liston_dir/'swarm/swarm_log'
swarm_log_dir.mkdir(exist_ok=True, parents=True)

In [102]:
subjects + ['247183']

['24563', '24573', '24704', '24718', '247183', '24740', '24742', '247183']

In [139]:
cmds

['bash /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/func_smooth_subcort_concat.sh /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline sub-24563 /data/EDB/TMSpilot/liston',
 'bash /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/func_smooth_subcort_concat.sh /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline sub-24573 /data/EDB/TMSpilot/liston',
 'bash /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/func_smooth_subcort_concat.sh /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline sub-24704 /data/EDB/TMSpilot/liston',
 'bash /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipelin

In [ ]:
/data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/func_vol2surf.sh sub-24546 /data/EDB/TMSpilot/liston /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/config/CiftiList.txt 1

In [59]:
nthreads = 20
cmds = []
for sub in ['24546']:
    cmd = [
        'bash',
        func_script_path,
        liston_dir.as_posix(),
        f'sub-{sub}',
        f'{nthreads}'
    ]
    cmds.append(' '.join(cmd))

In [60]:
cmds

['bash /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/func_preproc+denoise_ME-fMRI_wrapper.sh /data/EDB/TMSpilot/liston sub-24546 20']

In [61]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'liston_func'
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'liston_func'
    jobid = ! swarm -f {swarm_cmd_file} -g 250 -t 22 --gres=lscratch:200 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 48:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm
    print(jobid)

['12532787']


### smooth the data and do subcortical smoothing
cd  /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/Res0urces/; matlab -nodesktop -nosplash -r "pfm_example_use; exit"

In [62]:
smooth_script_path = '/data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/func_smooth_subcort_concat.sh'
medir="/data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline"


In [26]:
liston_dir

PosixPath('/data/EDB/TMSpilot/liston')

In [63]:
nthreads = 20
cmds = []
for sub in ['24546']:
    cmd = [
        'bash',
        smooth_script_path,
        medir,
        f'sub-{sub}',
        liston_dir.as_posix(),
    ]
    cmds.append(' '.join(cmd))

In [28]:
cmds

['bash /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline/func_smooth_subcort_concat.sh /data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/MultiEchofMRI-Pipeline sub-24718 /data/EDB/TMSpilot/liston']

In [64]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'liston_smooth'
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'liston_smooth'
    jobid = ! swarm -f {swarm_cmd_file} -g 140 -t 22 --gres=lscratch:200 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 4:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition norm,quick
    print(jobid)

['12607103']


 ## split sessions

In [2]:
split_subjs = ['247183', '24740', '24742', '245461', '245462']

In [3]:
data_dir = Path('/data/EDB/TMSpilot/liston/')
fd_thresh = 0.3

In [4]:
subjects = ['24546', '24563', '24573', '24704', '24718', '24740', '24742', '247183', '245461', '245462']

In [8]:
jobs = []
for subject in subjects: #['24563', '24573', '24704', '24718', '24740', '24742']:
    sub_concat_dir = data_dir / f'sub-{subject}/func/rest/ConcatenatedCiftis'
    fd = sio.loadmat(sub_concat_dir / 'FD.mat')['FD'].squeeze()
    scanid = sio.loadmat(sub_concat_dir / 'ScanIdx.mat')['ScanIdx'].squeeze()
    scanid = pd.DataFrame(scanid, columns=['session', 'run'])
    scanid['ses_run'] = scanid.session.astype(str) + scanid.run.astype(str)
    run_starts = scanid.ses_run.drop_duplicates().index.values
    dummy_idxs = np.hstack([run_starts, run_starts + 1, run_starts +2, run_starts + 3])
    fd_mask = fd < fd_thresh
    # drop first 4 frames from each run
    fd_mask[dummy_idxs] = False
    scanid['fd_mask'] = fd_mask

    for smoothing in [2.55, 5]:
        for nruns in ['all', 2]:
            concat_nii =  sub_concat_dir / f'Rest_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing{smoothing}.dtseries.nii'
        
        for session in [1,2]:
            ses_concat_dir = sub_concat_dir.parent / f'session_{session}/concatenated'
            ses_concat_dir.mkdir(exist_ok=True, parents=True)
            if nruns == 'all':
                continue
                ses_concat_name = concat_nii.parts[-1].replace('Rest_', f'Rest_session-{session}_')
            elif nruns == 2:
                ses_concat_name = concat_nii.parts[-1].replace('Rest_', f'Rest_session-{session}_2runs+')
            ses_concat_out = ses_concat_dir / ses_concat_name
            cmd = [
                'wb_command',
                '-cifti-merge',
                ses_concat_out.as_posix(),
                '-cifti',
                concat_nii.as_posix(),
            ]
            if nruns == 'all':
                continue
                if subject in split_subjs:
                    if session == 1:
                        cmd += (' '.join([f'-column {trn + 1}' for trn in scanid.loc[(scanid.run < 4) & scanid.fd_mask].index.values])).split(' ')
                    elif session == 2:
                        cmd += (' '.join([f'-column {trn + 1}' for trn in scanid.loc[(scanid.run >= 4) & scanid.fd_mask].index.values])).split(' ')
                else:
                    cmd += (' '.join([f'-column {trn + 1}' for trn in scanid.loc[(scanid.session == session) & scanid.fd_mask].index.values])).split(' ')

            elif nruns == 2:
                if subject in split_subjs:
                    if session == 1:
                        cmd += (' '.join([f'-column {trn + 1}' for trn in scanid.loc[(scanid.run <= 2) & scanid.fd_mask].index.values])).split(' ')
                    elif session == 2:
                        cmd += (' '.join([f'-column {trn + 1}' for trn in scanid.loc[(scanid.run >= 4) & scanid.fd_mask & (scanid.run < 6)].index.values])).split(' ')
                else:
                    cmd += (' '.join([f'-column {trn + 1}' for trn in scanid.loc[(scanid.session == session) & scanid.fd_mask & (scanid.run <= 2)].index.values])).split(' ')
            
            jobs.append(delayed(subprocess.run)(cmd))


In [10]:
Parallel(n_jobs=20, verbose=10)(jobs)

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    9.3s
[Parallel(n_jobs=20)]: Done   6 out of  40 | elapsed:   10.3s remaining:   58.5s
[Parallel(n_jobs=20)]: Done  11 out of  40 | elapsed:   15.2s remaining:   40.2s
[Parallel(n_jobs=20)]: Done  16 out of  40 | elapsed:   15.8s remaining:   23.6s
[Parallel(n_jobs=20)]: Done  21 out of  40 | elapsed:   25.2s remaining:   22.8s
[Parallel(n_jobs=20)]: Done  26 out of  40 | elapsed:   27.1s remaining:   14.6s
[Parallel(n_jobs=20)]: Done  31 out of  40 | elapsed:   30.1s remaining:    8.7s
[Parallel(n_jobs=20)]: Done  36 out of  40 | elapsed:   36.9s remaining:    4.1s
[Parallel(n_jobs=20)]: Done  40 out of  40 | elapsed:   40.8s finished


[CompletedProcess(args=['wb_command', '-cifti-merge', '/data/EDB/TMSpilot/liston/sub-24546/func/rest/session_1/concatenated/Rest_session-1_2runs+OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing2.55.dtseries.nii', '-cifti', '/data/EDB/TMSpilot/liston/sub-24546/func/rest/ConcatenatedCiftis/Rest_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing2.55.dtseries.nii', '-column', '5', '-column', '6', '-column', '7', '-column', '8', '-column', '9', '-column', '10', '-column', '11', '-column', '12', '-column', '13', '-column', '14', '-column', '15', '-column', '16', '-column', '17', '-column', '18', '-column', '19', '-column', '20', '-column', '21', '-column', '22', '-column', '23', '-column', '24', '-column', '25', '-column', '26', '-column', '27', '-column', '28', '-column', '29', '-column', '30', '-column', '31', '-column', '32', '-column', '33', '-column', '34', '-column', '35', '-column', '36', '-column', '37', '-column', '38', '-column', '39', '-column', '40', '

In [36]:
scanid.session == session

0        True
1        True
2        True
3        True
4        True
        ...  
2875    False
2876    False
2877    False
2878    False
2879    False
Name: session, Length: 2880, dtype: bool

In [44]:
scanid.loc[(scanid.session == session) & scanid.fd_mask].index.values

array([   4,    5,    6, ..., 1437, 1438, 1439])

In [29]:
fd_mask.nonzero()[0][(scanid.session == session).values]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 2793 but corresponding boolean dimension is 2880

In [26]:
len(fd_mask)

2880

In [28]:
len((scanid.session == session).values)

2880

In [20]:
subject

'24546'

In [19]:
len(scanid)

2880

In [21]:
fd_mask[(scanid.session == 1).values]

array([False, False, False, ...,  True,  True,  True])

In [22]:
fd_mask[(scanid.session == 2).values]

array([False, False, False, ...,  True,  True,  True])

In [23]:
scanid

,session,run,ses_run
0,1,1,11
1,1,1,11
2,1,1,11
3,1,1,11
4,1,1,11
...,...,...,...
2875,2,6,26
2876,2,6,26
2877,2,6,26
2878,2,6,26


In [11]:
cmds

[['wb_command',
  '-cifti-merge',
  '/data/EDB/TMSpilot/liston/sub-24546/func/rest/session_1/concatenated/Rest_session-1_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing2.55.dtseries.nii',
  '-cifti',
  '/data/EDB/TMSpilot/liston/sub-24546/func/rest/ConcatenatedCiftis/Rest_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing2.55.dtseries.nii',
  '-column',
  '5',
  '-column',
  '6',
  '-column',
  '7',
  '-column',
  '8',
  '-column',
  '9',
  '-column',
  '10',
  '-column',
  '11',
  '-column',
  '12',
  '-column',
  '13',
  '-column',
  '14',
  '-column',
  '15',
  '-column',
  '16',
  '-column',
  '17',
  '-column',
  '18',
  '-column',
  '19',
  '-column',
  '20',
  '-column',
  '21',
  '-column',
  '22',
  '-column',
  '23',
  '-column',
  '24',
  '-column',
  '25',
  '-column',
  '26',
  '-column',
  '27',
  '-column',
  '28',
  '-column',
  '29',
  '-column',
  '30',
  '-column',
  '31',
  '-column',
  '32',
  '-column',
  '33',
  '-column',
  '34',
  '

In [189]:
nb.load(ses_concat_out).shape

(460, 82428)

In [179]:
subprocess.run(cmd)

5770

## Make FSLR images, Dconn for separate sessions, run template matching

In [74]:
def create_fslr32k_sample(subject, datadir, medir, hcpdir, session=1, run=1, overwrite=False):
    """
    Create actual fslr32k space examples from 
    Session and run default to 1 because we only need to make a single sample image in order to run the transformation.
    """
    subdir = datadir / f'sub-{subject}'
    
    
    rundir = subdir / f'func/rest/session_{session}/run_{run}'
    acpc_rest = rundir / 'Rest_OCME+MEICA+MGTR.nii.gz'
    ocifti_rest = rundir / 'Rest_OCME+MEICA+MGTR.dtseries.nii'
    nonlin_rest = rundir / 'Rest_OCME+MEICA+MGTR_nonlin.nii.gz'
    rois = hcpdir / 'global/templates/91282_Greyordinates/Atlas_ROIs.2.nii.gz'
    mni152_2mm = medir / 'Res0urces/FSL/MNI152_T1_2mm.nii.gz'
    
    warp = subdir / 'anat/MNINonLinear/xfms/acpc_dc2standard.nii.gz'        
    
    if not nonlin_rest.exists() or overwrite:
        # create a functional volume in MNI space
        cmd = [
            'applywarp',
            '-i',
            acpc_rest.as_posix(),
            '-r',
            mni152_2mm.as_posix(),
            '-o',
            nonlin_rest.as_posix(),
            '-w',
            warp.as_posix(),
            '--interp=trilinear'
        ]
        subprocess.run(cmd, check=True)
    #     # trim all but first TR to save space

    #     cmd = [
    #         'fslroi',
    #         nonlin_rest,
    #         nonlin_rest,
    #         '0', 
    #         '1'
    #     ]
    #     subprocess.run(cmd, check=True)

    left_metric = rundir / 'lh.32k_fs_LR.shape.gii'
    right_metric = rundir / 'rh.32k_fs_LR.shape.gii'
    if not (left_metric.exists() & right_metric.exists()) or overwrite:

        # split out giftis
        cmd = [
            'wb_command',
            '-cifti-separate',
            ocifti_rest.as_posix(),
            'COLUMN',
            '-metric',
            'CORTEX_LEFT',
            left_metric.as_posix(),
            '-metric',
            'CORTEX_RIGHT',
            right_metric.as_posix()
        ]
        subprocess.run(cmd, check=True)


    # create a cifti that's actually in fslr
    tr_file = rundir / 'TR.txt'
    tr = float(tr_file.read_text().strip())
    fslr_cifti_example = rundir / 'Rest_OCME+MEICA+MGTR_fsLR32k.dtseries.nii'
    left_roi = subdir / f'anat/MNINonLinear/fsaverage_LR32k/sub-{subject}.L.atlasroi.32k_fs_LR.shape.gii'
    right_roi = subdir / f'anat/MNINonLinear/fsaverage_LR32k/sub-{subject}.R.atlasroi.32k_fs_LR.shape.gii'
    if not fslr_cifti_example.exists() or overwrite:
        cmd = [
            'wb_command',
            '-cifti-create-dense-timeseries',
            fslr_cifti_example.as_posix(),
            '-volume',
            nonlin_rest.as_posix(),
            rois.as_posix(),
            '-left-metric',
            left_metric.as_posix(),
            '-roi-left',
            left_roi.as_posix(),
            '-right-metric',
            right_metric.as_posix(),
            '-roi-right',
            right_roi.as_posix(),
            '-timestep',
            str(tr)
        ]
        subprocess.run(cmd, check=True)
    return fslr_cifti_example

def _nlresample_wrapper(acpc_t1w, source_cifti, fslr_example, warp, standard, outpath, overwrite=False):
    if not Path(outpath).exists() or overwrite:
        cmd = [
            'wb_command',
            '-cifti-resample',
            source_cifti.as_posix(),
            'COLUMN',
            fslr_example.as_posix(),
            'COLUMN',
            'ADAP_BARY_AREA',
            'TRILINEAR',
            outpath.as_posix(),
            '-warpfield',
            warp.as_posix(),
            '-fnirt',
            acpc_t1w.as_posix()
        ]
        subprocess.run(cmd, check=True)
    return outpath

def resample_procced_to_realfslr32(subject, smoothing, datadir, medir, hcpdir, overwrite=False):
    datadir = Path(datadir)
    medir = Path(medir)
    hcpdir = Path(hcpdir)
    subdir = datadir / f'sub-{subject}'
    fslr_example = create_fslr32k_sample(subject, datadir, medir, hcpdir, overwrite=False)
    acpc_t1w = subdir / 'anat/T1w/T1w_acpc_dc_restore.nii.gz'
    warp = subdir / 'anat/MNINonLinear/xfms/acpc_dc2standard.nii.gz'
    mni152_2mm = medir / 'Res0urces/FSL/MNI152_T1_2mm.nii.gz'
    catted_dir = subdir / 'func/rest/ConcatenatedCiftis'
    procd_cifti = catted_dir / f'Rest_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing{smoothing}.dtseries.nii'
    fslr_procd_cifti = catted_dir / f'Rest_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing{smoothing}_fslr32k.dtseries.nii'
    
    _ = _nlresample_wrapper(acpc_t1w, procd_cifti, fslr_example, warp, mni152_2mm, fslr_procd_cifti, overwrite=overwrite)
    return fslr_procd_cifti

In [75]:
subjects = ['24563','24546', '245461', '245462', '24573', '24704', '24718', '247183', '24740', '24742']
datadir = Path('/data/EDB/TMSpilot/liston')
medir = Path('/data/MLDSST/nielsond/target_test/other_repos/Liston-Laboratory-MultiEchofMRI-Pipeline/')
hcpdir = Path('/data/MLDSST/nielsond/target_test/other_repos/HCPpipelines')
fd_thresh = 0.3

In [76]:
swarm_cmd_dir = datadir/'swarm/swarm_cmds'
swarm_cmd_dir.mkdir(exist_ok=True, parents=True)
swarm_log_dir = datadir/'swarm/swarm_log'
swarm_log_dir.mkdir(exist_ok=True, parents=True)

In [77]:
jobs = []
for subject in subjects:
    for smoothing in [2.55, 5]:
        jobs.append(delayed(resample_procced_to_realfslr32)(subject, smoothing, datadir, medir, hcpdir, overwrite=True))
        #resample_procced_to_realfslr32(subject, smoothing, datadir, medir, hcpdir, overwrite=True)
print(len(jobs))


20


In [78]:
res = Parallel(n_jobs=14, verbose=10)(jobs)

[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   2 out of  20 | elapsed:   32.3s remaining:  4.8min
[Parallel(n_jobs=14)]: Done   5 out of  20 | elapsed:   44.9s remaining:  2.2min
[Parallel(n_jobs=14)]: Done   8 out of  20 | elapsed:   49.4s remaining:  1.2min
[Parallel(n_jobs=14)]: Done  11 out of  20 | elapsed:  1.4min remaining:  1.2min
[Parallel(n_jobs=14)]: Done  14 out of  20 | elapsed:  1.6min remaining:   40.7s

SEVERE: error closing compressed file '/data/EDB/TMSpilot/liston/sub-24740/func/rest/session_1/run_1/Rest_OCME+MEICA+MGTR_nonlin.nii.gz'
Method: virtual caret::ZFileImpl::~ZFileImpl()
Location: CaretBinaryFile.cxx:276


While running:
/usr/local/apps/connectome-workbench/1.5.0/workbench/bin_rh_linux64/../exe_rh_linux64/wb_command -cifti-create-dense-timeseries /data/EDB/TMSpilot/liston/sub-24740/func/rest/session_1/run_1/Rest_OCME+MEICA+MGTR_fsLR32k.dtseries.nii -volume /data/EDB/TMSpilot/liston/sub-24740/func/r

CalledProcessError: Command '['wb_command', '-cifti-create-dense-timeseries', '/data/EDB/TMSpilot/liston/sub-24740/func/rest/session_1/run_1/Rest_OCME+MEICA+MGTR_fsLR32k.dtseries.nii', '-volume', '/data/EDB/TMSpilot/liston/sub-24740/func/rest/session_1/run_1/Rest_OCME+MEICA+MGTR_nonlin.nii.gz', '/data/MLDSST/nielsond/target_test/other_repos/HCPpipelines/global/templates/91282_Greyordinates/Atlas_ROIs.2.nii.gz', '-left-metric', '/data/EDB/TMSpilot/liston/sub-24740/func/rest/session_1/run_1/lh.32k_fs_LR.shape.gii', '-roi-left', '/data/EDB/TMSpilot/liston/sub-24740/anat/MNINonLinear/fsaverage_LR32k/sub-24740.L.atlasroi.32k_fs_LR.shape.gii', '-right-metric', '/data/EDB/TMSpilot/liston/sub-24740/func/rest/session_1/run_1/rh.32k_fs_LR.shape.gii', '-roi-right', '/data/EDB/TMSpilot/liston/sub-24740/anat/MNINonLinear/fsaverage_LR32k/sub-24740.R.atlasroi.32k_fs_LR.shape.gii', '-timestep', '2.5']' returned non-zero exit status 255.

In [82]:
[rr.exists() for rr in res]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [17]:
subject='sub-24563'
smoothing=2.55

concat_dir = data_dir / f'{subject}/func/rest/ConcatenatedCiftis'
fd_thresh = 0.3

In [18]:
concat_nii = sorted(data_dir.glob(f'{subject}/func/rest/ConcatenatedCiftis/Rest_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing{smoothing}.dtseries.nii'))[0]
fd = sio.loadmat(sorted(data_dir.glob(f'{subject}/func/rest/ConcatenatedCiftis/FD.mat'))[0])['FD'].squeeze()
scanid = sio.loadmat(sorted(data_dir.glob(f'{subject}/func/rest/ConcatenatedCiftis/ScanIdx.mat'))[0])['ScanIdx'].squeeze()

In [19]:
scanid

array([[1, 1],
       [1, 1],
       [1, 1],
       ...,
       [2, 2],
       [2, 2],
       [2, 2]], dtype=uint8)

In [22]:
concat_dir / 'FD.mat'

PosixPath('/data/EDB/TMSpilot/liston/24563/func/rest/ConcatenatedCiftis/FD.mat')

In [29]:
subject

'247183'

In [30]:
scanid

array([[1, 1],
       [1, 1],
       [1, 1],
       ...,
       [1, 6],
       [1, 6],
       [1, 6]], dtype=uint8)

In [12]:
split_subjs = ['247183', '24740', '24742']

In [125]:
subjects

['24563', '24573', '24704', '24718', '247183', '24740', '24742']

In [126]:
cmds = []
for subject in ['24563', '24573', '24704', '24718', '24740', '24742']:
    concat_dir = data_dir / f'sub-{subject}/func/rest/ConcatenatedCiftis'

    fd = sio.loadmat((concat_dir / 'FD.mat').as_posix())['FD'].squeeze()
    scanid = sio.loadmat((concat_dir / 'ScanIdx.mat').as_posix())['ScanIdx'].squeeze()
    for session in [1,2]:
        weights = (fd < fd_thresh).astype(int)
        # deal with 247183
        if subject in split_subjs:
            if session == 1:
                weights[scanid[:, 1] > 3] = 0
            else:
                weights[scanid[:, 1] <= 3] = 0
        else:
            # 0 weights for session mask
            weights[scanid[:, 0] != session] = 0
        dconn_sess_dir = data_dir / f'sub-{subject}/func/rest/session_{session}/conn'
        dconn_sess_dir.mkdir(exist_ok=True, parents=True)

        weight_file = dconn_sess_dir / 'weights.txt'
        weight_file.write_text(' ' .join(list(weights.astype(int).astype(str))) + '\n')
        for smoothing in [2.55, 5]:
            concat_nii = concat_dir / f'Rest_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing{smoothing}_fslr32k.dtseries.nii'

            dconn_res = dconn_sess_dir / concat_nii.parts[-1].replace("Rest", f"Rest_session-{session}").replace('.dtseries', '.dconn')
            cmd = [
                'wb_command',
                '-cifti-correlation',
                concat_nii.as_posix(),
                dconn_res.as_posix(),
                '-weights',
                weight_file.as_posix(),
                '-mem-limit',
                '100'
            ]
            cmds.append(' '.join(cmd))

In [84]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'liston_dconn'
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'liston_dconn'
    jobid = ! swarm -f {swarm_cmd_file} -g 140 -t 22 --gres=lscratch:200 --module matlab,freesurfer/6.0,fsl,simnibs/4.0,connectome-workbench,openblas  --time 4:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition quick,norm
    print(jobid)

['10892803']


In [86]:
template_matching_script_path = Path('/data/MLDSST/nielsond/target_test/other_repos/template_matching_container/run_once_biowulf.sh')

In [93]:
cmds = []
for subject in subjects:
    concat_dir = data_dir / f'sub-{subject}/func/rest/ConcatenatedCiftis'

    fd = sio.loadmat((concat_dir / 'FD.mat').as_posix())['FD'].squeeze()
    scanid = sio.loadmat((concat_dir / 'ScanIdx.mat').as_posix())['ScanIdx'].squeeze()
    for session in [1,2]:
        weights = (fd < fd_thresh).astype(int)
        # deal with 247183
        if subject == '247183':
            if session == 1:
                weights[scanid[:, 1] > 3] = 0
            else:
                weights[scanid[:, 1] <= 3] = 0
        else:
            # 0 weights for session mask
            weights[scanid[:, 0] != session] = 0
        dconn_sess_dir = data_dir / f'sub-{subject}/func/rest/session_{session}/conn'
        dconn_sess_dir.mkdir(exist_ok=True, parents=True)

        weight_file = dconn_sess_dir / 'weights.txt'
        weight_file.write_text(' ' .join(list(weights.astype(int).astype(str))) + '\n')
        for smoothing in [2.55, 5]:
            concat_nii = concat_dir / f'Rest_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing{smoothing}_fslr32k.dtseries.nii'

            dconn_res = dconn_sess_dir / concat_nii.parts[-1].replace("Rest", f"Rest_session-{session}").replace('.dtseries', '.dconn')
            template_matching_dir = dconn_sess_dir.parent / f'template_matching_smoothing-{smoothing}'
            template_matching_dir.mkdir(exist_ok=True)
            if dconn_res.exists():
                cmd = [
                    template_matching_script_path.as_posix(),
                    template_matching_script_path.parent.as_posix(),
                    dconn_res.as_posix(),
                    template_matching_dir.as_posix()
                ]
                cmds.append(' '.join(cmd))


In [94]:
cmds

['/data/MLDSST/nielsond/target_test/other_repos/template_matching_container/run_once_biowulf.sh /data/MLDSST/nielsond/target_test/other_repos/template_matching_container /data/EDB/TMSpilot/liston/sub-24563/func/rest/session_1/conn/Rest_session-1_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing2.55_fslr32k.dconn.nii /data/EDB/TMSpilot/liston/sub-24563/func/rest/session_1/template_matching_smoothing-2.55',
 '/data/MLDSST/nielsond/target_test/other_repos/template_matching_container/run_once_biowulf.sh /data/MLDSST/nielsond/target_test/other_repos/template_matching_container /data/EDB/TMSpilot/liston/sub-24563/func/rest/session_1/conn/Rest_session-1_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing5_fslr32k.dconn.nii /data/EDB/TMSpilot/liston/sub-24563/func/rest/session_1/template_matching_smoothing-5',
 '/data/MLDSST/nielsond/target_test/other_repos/template_matching_container/run_once_biowulf.sh /data/MLDSST/nielsond/target_test/other_repos/template_matching_

In [96]:
if len(cmds) > 0:
    swarm_cmd_file = swarm_cmd_dir / 'template_matching'
    swarm_cmd_file.write_text('\n'.join(cmds))
    run_name = 'template_matching'
    jobid = ! swarm -f {swarm_cmd_file} -g 140 -t 22 --gres=lscratch:200 --module singularity  --time 4:00:00 --logdir {swarm_log_dir} --job-name {run_name} --partition quick,norm
    print(jobid)

['10894961']


In [28]:
!ls /data/EDB/TMSpilot/liston/sub-247183/func/rest/

AverageSBref  ConcatenatedCiftis  session_1


In [60]:
data_dir

PosixPath('/data/EDB/TMSpilot/liston')

In [50]:
concat_nii = sorted(data_dir.glob(f'{subject}/func/rest/ConcatenatedCiftis/Rest_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing{smoothing}.dtseries.nii'))[0]
fd = sio.loadmat(sorted(data_dir.glob(f'{subject}/func/rest/ConcatenatedCiftis/FD.mat'))[0])['FD'].squeeze()
scanid = sio.loadmat(sorted(data_dir.glob(f'{subject}/func/rest/ConcatenatedCiftis/ScanIdx.mat'))[0])['ScanIdx'].squeeze()

'Rest_session-1_OCME+MEICA+MGTR_Concatenated+SubcortRegression+SpatialSmoothing2.55.dconn.nii'

In [35]:
nb.load(concat_nii[0]).shape

(960, 82428)

'/data/MLDSST/nielsond/target_test/data/cornell/